In [54]:
from polybinder.system import System, Initializer
from polybinder.simulate import Simulation
import mbuild as mb
import hoomd
from mbuild.formats.hoomd_forcefield import to_hoomdsnapshot
import numpy as np

In [55]:
import warnings
warnings.filterwarnings("ignore")

In [56]:
## set parameters
dt = 0.0003
tau_kT = 10
kT = 1.5

In [57]:
system_params = System(density=0.01,
                    molecule="PPS",
                    polymer_lengths=1,
                    n_compounds=10,
                    para_weight=1.0,
                   )

In [58]:
PPS_system_saved = Initializer(system=system_params,
                        system_type='stack',
                        remove_hydrogens=True,
                        n_rigid_beads=system_params.polymer_lengths[0] * system_params.n_compounds[0],
                        rigid_bead_len=7,
                        rigid_bead_names=['R1', 'R2'],
                        separation=0.6,
                        forcefield="opls",
                        save_parmed=False
                              )

Removing hydrogens and adjusting heavy atoms
---------------------------------------------------------------
make rigid snap...


In [59]:
import os
from datetime import datetime
log_dir = "/home/marjanalbooyeh/logs/pps_rigid/"
now = datetime.now()
log_path = os.path.join(log_dir, now.strftime("%Y-%m-%d-%H:%M:%S")) + '/'


if not os.path.exists(log_path):
    os.mkdir(log_path)

In [60]:
simulation = Simulation(system=PPS_system_saved, dt=dt, tau_kt=tau_kT, logs_path=log_path, gsd_write=1000, log_write=1000)

Processing LJ and QQ
No charged groups found, ignoring electrostatics
Processing 1-4 interactions, adjusting neighborlist exclusions
Processing harmonic bonds
Processing harmonic angles
Processing RB torsions
body_name:  R2
body_name:  R2
body_name:  R2
body_name:  R2
body_name:  R2
body_name:  R2
body_name:  R2
body_name:  R2
body_name:  R2
body_name:  R2


In [123]:
simulation.sim.operations.integrator.forces[0].nlist.exclusions.extend(['body'])

In [61]:
for i, force in enumerate(simulation.sim.operations.integrator.forces):
#     if isinstance(force, hoomd.md.pair.LJ):
#         for t in snapshot.particles.types:
#             for r in ['R1', 'R2']:
#                 force.params[(r, t)] = dict(epsilon=0, sigma=0)
#                 force.r_cut[(r, t)] = 0
    if isinstance(force, hoomd.md.special_pair.LJ):
        for k in force.params.keys():
            force.params[k] = dict(epsilon=0., sigma=0.)
    if isinstance(force, hoomd.md.bond.Harmonic):
        for k in force.params.keys():
            force.params[k] = dict(k=0, r0=0)
    if isinstance(force, hoomd.md.angle.Harmonic):
        for k in force.params.keys():
            force.params[k] = dict(t0=0, k=0)
    if isinstance(force, hoomd.md.dihedral.Harmonic):
        for k in force.params.keys():
            force.params[k] = dict(k=0, d=0, n=0, phi0=0)

In [12]:
simulation.quench(n_steps=1, kT=kT)

In [62]:
simulation.quench(n_steps=2e6, kT=kT)

*Warning*: angle.harmonic: specified K <= 0
*Warning*: angle.harmonic: specified t_0 <= 0
*Warning*: angle.harmonic: specified K <= 0
*Warning*: angle.harmonic: specified t_0 <= 0


In [42]:
log_path

'/home/marjanalbooyeh/logs/pps_rigid/2023-03-09-13:40:38/'

In [11]:
simulation.sim.timestep

10000

In [44]:

simulation.sim.operations.writers[-1].file.close()

### prepare data

In [18]:
import h5py
import numpy as np
import pandas as pd
# log_path = '/home/marjanalbooyeh/logs/pps_rigid/2023-02-22-14:22:25/'

In [19]:
rigid_traj_path = log_path + 'rigid_traj.h5'
f = h5py.File(rigid_traj_path, 'r')

In [20]:
traj_df = pd.DataFrame()
for frame in f.keys():
    dset = dict(f[frame])
    dset = {key: np.array(value) for key, value in dset.items()}
    dset['timestep'] = f[frame].attrs['timestep']
    traj_df = traj_df.append(dset, ignore_index=True)

In [21]:
f.close()

In [22]:
traj_df.head()

,net_force,net_torque,orientation,position,timestep
0,"[[2.3195248836672344, -0.016961836506932768, 0...","[[-0.0016085701309671166, -0.25658213305082583...","[[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0]]","[[-1.0034769895991462, -0.0010186344033643584,...",0
1,"[[11.760050725530498, 17.9702858482377, 28.403...","[[22.170426593418913, -13.609429331768359, -2....","[[-0.3553022009380696, -0.5269938769149056, -0...","[[0.06090373476765563, -0.1774404145755944, 0....",9000
2,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]","[[-2.7338835405579798e-17, 1.7412964516475005e...","[[-0.6926463069144344, -0.5109427186714979, 0....","[[-4.608168493579463, 2.177932632556924, 0.706...",99000
3,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]","[[-2.2156687834277988e-16, -4.147920342704309e...","[[-0.30886829650599484, 0.9139738003793408, 0....","[[1.5617890874290794, -3.6638066667130356, 1.8...",999000
4,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]","[[2.2546322990014966e-16, -1.8903318987168946e...","[[-0.22783384176885263, 0.7715583648236748, 0....","[[1.855570306598415, -4.017858712899107, 1.506...",1000000


In [76]:
positions = np.asarray(traj_df["position"].tolist())
orientations = np.asarray(traj_df["orientation"].tolist())

In [77]:
rel_positions = np.zeros_like(positions)
rel_positions[:, 0, :] = positions[:, 0, :] - positions[:, 1, :]
rel_positions[:, 1, :] = positions[:, 1, :] - positions[:, 0, :]

In [78]:
rel_orientations = np.zeros_like(orientations)
rel_orientations[:, 0, :] = orientations[:, 0, :] - orientations[:, 1, :]
rel_orientations[:, 1, :] = orientations[:, 1, :] - orientations[:, 0, :]

In [79]:
columns = [
    "position",
    "orientation",
    "net_force",
    "net_torque",
]

import pandas as pd
new_traj_df = pd.DataFrame(columns=columns)

In [80]:
new_traj_df["position"] = rel_positions.tolist()
new_traj_df["orientation"] = rel_orientations.tolist()
new_traj_df["net_force"] = traj_df["net_force"].tolist()
new_traj_df["net_torque"] = traj_df["net_torque"].tolist()

In [81]:
new_traj_df.head()

,position,orientation,net_force,net_torque
0,"[[-2.244924096618746, 0.0, 0.0], [2.2449240966...","[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]","[[0.9619755684694091, -0.00593004320234769, 0....","[[-0.0005679807434297857, -0.12818891765945012..."
1,"[[1.6710277973776102, 1.091021141733464, 1.108...","[[-0.7789667048026656, 0.9322633644617142, 0.3...","[[-4.444279903935069, -2.4822420121960818, -1....","[[-0.4545146836770524, 1.1319537690556642, -0...."
2,"[[2.8853238552283083, 8.54345629425984, 7.9942...","[[0.532382931775278, -0.6795385700817432, -0.4...","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]","[[3.17822301809656e-09, -2.0108714693378715e-0..."
3,"[[4.504938525363006, -2.556042073814547, 8.119...","[[-0.31869885059382874, -0.19245421931668943, ...","[[-1.4210854715202004e-14, 3.552713678800501e-...","[[2.891573203100961e-09, -4.5065284837164654e-..."
4,"[[4.5721037279064145, -2.5121251836981173, 8.0...","[[-0.4168202372222529, -0.20519349667960213, 0...","[[0.0, -2.1316282072803006e-14, 0.0], [-2.8421...","[[1.783567071811376e-09, 8.644800431056865e-10..."


In [84]:
traj_df.to_pickle("/home/marjanalbooyeh/code/cme-lab/ML_datasets/pps_rigid_Feb22_2/raw_data/pps_rigid_2.pkl")